In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
twitter = pd.read_csv('data/gender-classifier-DFE-791531.csv', encoding='iso-8859-1' )

The dataset contains the following fields:

**unit_id:** a unique id for user

**golden:** whether the user was included in the gold standard for the model; TRUE or FALSE

**unit_state:** state of the observation; one of finalized (for contributor-judged) or golden (for gold standard observations)

**trusted_judgments:** number of trusted judgments (int); always 3 for non-golden, and what may be a unique id for gold standard observations

**last_judgment_at:** date and time of last contributor judgment; blank for gold standard observations

**gender:** one of male, female, or brand (for non-human profiles)

**gender:confidence:** a float representing confidence in the provided gender

**profile_yn:** "no" here seems to mean that the profile was meant to be part of the dataset but was not available when contributors went to judge it

**profile_yn:confidence:** confidence in the existence/non-existence of the profile

**created:** date and time when the profile was created

**description:** the user's profile description

**fav_number:** number of tweets the user has favorited

**gender_gold:** if the profile is golden, what is the gender?

**link_color:** the link color on the profile, as a hex value

**name:** the user's name

**profile_yn_gold:** whether the profile y/n value is golden

**profileimage:** a link to the profile image

**retweet_count:** number of times the user has retweeted (or possibly, been retweeted)

**sidebar_color:** color of the profile sidebar, as a hex value

**text:** text of a random one of the user's tweets

**tweet_coord:** if the user has location turned on, the coordinates as a string with the format "[latitude, longitude]"

**tweet_count:** number of tweets that the user has posted

**tweet_created:** when the random tweet (in the text column) was created

**tweet_id:** the tweet id of the random tweet

**tweet_location:** location of the tweet; seems to not be particularly normalized

**user_timezone:** the timezone of the user

In [7]:
twitter.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [11]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr